In [4]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

  Using cached mediapipe_model_maker-0.2.1.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow-2.15.1-cp39-cp39-win_amd64.whl.metadata (3.4 kB)
  Using cached tensorflow_model_optimization-0.7.5-py2.py3-none-any.whl.metadata (914 bytes)
  Using cached tensorflow_text-2.10.0-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached tf_models_official-2.15.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached tensorflow_intel-2.15.1-cp39-cp39-win_amd64.whl.metadata (4.9 kB)
  Using cached ml_dtypes-0.3.2-cp39-cp39-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.7-cp39-cp39-win_amd64.whl.metadata (541 bytes)
  Using cached wrapt-1.14.1-cp39-cp39-win_amd64.whl.metadata (6.7 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached google_auth_oauthlib-1.2.2-py3-none-any.whl.metadata (2.7 kB)
  

# **Import the Required Models**

In [5]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'google.colab'

# **Add the Downloaded dataset to your google drives `My Drive` Section. Then use Below Code to connect it to this Project**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
my_folder_path = '/content/drive/MyDrive/sign_language_dataset'

In [ ]:
print(my_folder_path)

In [ ]:
print(my_folder_path)
labels = []
for i in os.listdir(my_folder_path):
  if os.path.isdir(os.path.join(my_folder_path, i)):
    labels.append(i)
for label in labels:
  print(label + "\n")

In [ ]:
print(len(labels))

# **From Below Run All the Steps to Train and then Download the Trained Model**

In [ ]:
NUM_EXAMPLES = 10

for label in labels:
  label_dir = os.path.join(my_folder_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=my_folder_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

In [ ]:
model.export_model()
!ls exported_model

In [ ]:
files.download('exported_model/gesture_recognizer.task')